In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# Data Acquision
def DataAq():
    train = pd.read_csv("../input/train.csv")
    test = pd.read_csv("../input/test.csv")
    return train,test# Data Acquision


In [ ]:
# Isolate the price from the data
def separate_price(train):
    target=train['price_doc']
    train.drop('price_doc',inplace=True,axis=1)
    return target

In [ ]:
# Replace the nan with the mean
def replace_nan_mean(feature):
    global combined
    combined[feature].fillna(combined[feature].mean(),inplace=True)

In [ ]:
# concatanate train and tests datasets into one dataset -> combined
def get_combined_data(train,test):
    global combined
    combined = train.append(test)
    combined.reset_index(inplace=True)
    combined.drop('index',inplace=True,axis=1)
    print (combined.shape)

In [ ]:
# Create dummies for non-numerical features
def p_generic(feature):
    global combined
    print ("Histogram of " +feature)
    print (combined[feature].value_counts())
    feature_dummies=pd.get_dummies(combined[feature],prefix=feature)
    combined=pd.concat([combined,feature_dummies],axis=1)
    combined.drop(feature,axis=1,inplace=True)

In [ ]:
# If nan perceptage of a feature is above 25% then drop this feature
def check_nan_percentage(feature):
    #print ((combined[feature].isnull().sum())/len(combined[feature]))
    if (combined[feature].isnull().sum())/len(combined[feature]) > 0.25:
        print (feature,"out")
        combined[feature].drop

In [ ]:
# If a feature is not numerical call p_generic for it (to make dummies)
def find_categoricals_and_make_dummies(feature):
    if (combined[feature].dtypes==object):
        print ('Categorical feature:',feature)
        p_generic(feature)

In [ ]:
# Change the date, keeping just the year
def change_date_format():
    combined['timestamp'] = pd.DatetimeIndex(combined['timestamp']).year

In [ ]:
# Scale all features
def scale_all_features():
    
    global combined
    
    features = list(combined.columns)
    #features.remove('PassengerId')
    combined[features] = combined[features].apply(lambda x: x/x.max(), axis=0)
    
    print ('Features scaled successfully !')

In [ ]:
# Recover train, test data from global data
def recover_train_test_target():
    global combined
    
    train = combined.ix[0:30470]
    test = combined.ix[30471:]
    
    return train,test   

In [ ]:
# grid search with cross-validation
def HT(targets,train_new, algorithm, parameters):
    c_v=StratifiedKFold(n_splits=5)
    c_v=c_v.get_n_splits(train_new, targets)
    scorer=make_scorer(mean_squared_error)
    grid_search = GridSearchCV(algorithm, param_grid=parameters, scoring=scorer, cv=c_v)

    grid_fit=grid_search.fit(train_new, targets.ravel())
    print ("Results for {}".format(algorithm))
    print (algorithm.__class__.__name__)
    print('Best score: {}'.format(grid_fit.best_score_))
    print('Best parameters: {}'.format(grid_fit.best_params_))
    pipeline=grid_search
    return pipeline.best_estimator_ 

In [ ]:
train,test=DataAq()
target=np.log(separate_price(train))
get_combined_data(train,test)
change_date_format()
for x in combined.columns:    
    check_nan_percentage(x)
for x in combined.columns:    
    find_categoricals_and_make_dummies(x)

for x in combined.columns:     
    replace_nan_mean(x)
    
scale_all_features()
r_train,r_test=recover_train_test_target()
reg_C=RandomForestRegressor()
reg_C_parameters={'n_estimators': [500], 'max_features':['sqrt']}   
pipeline3 = HT(target,r_train, reg_C, reg_C_parameters)
output3= np.exp(pipeline3.predict(r_test))
print (output3)

It seems to be a big difference between the score obtain here 0.23 and the score on the LeaderBoard 0.35. I am a newbie in this field :) , If you have any suggestion for improvement I am happy to hear them for improving myself.

Thanks for reading.